### 获取所有股票代码

In [1]:
import os
import json

# Define the updated function to modify the filenames as required
def get_ticker_with_prefix(path):
    tickers = []
    for f in os.listdir(path):
        if f.endswith('.csv'):
            # Extract numeric part
            numeric_part = ''.join(filter(str.isdigit, f))
            # Extract the letter part (.SZ or .SH), convert to lowercase and prepend to the numeric part
            letter_part = f.split('.')[1].lower()
            #ticker = letter_part + numeric_part
            ticker = numeric_part
            tickers.append(ticker)
    # Sort the list of IDs
    return sorted(tickers)

# Replace 'path_to_folder' with the actual path to your folder containing the CSV files
path_to_folder = 'data/raw-data/2024/2024'

# Get the modified stock ids
tickers_with_prefix = get_ticker_with_prefix(path_to_folder)

# Convert the list of modified stock ids to JSON format
json_content = json.dumps(tickers_with_prefix, indent=4)

# Replace 'path_to_json_file' with the actual path where you want to save the JSON file
path_to_json_file = 'data/tickers.json'

# Write the JSON content to a file
with open(path_to_json_file, 'w') as json_file:
    json_file.write(json_content)

### 按股票合并市值最高股票按年的数据

In [3]:
import os
import pandas as pd
import json


# 路径设置
json_file = 'data/top_cap_tickers.json'
data_dir = 'data/raw-data/'
merged_dir = 'data/by_stock_merged_top_caps/'
if not os.path.exists(merged_dir):
    os.makedirs(merged_dir)

# 读取股票代码列表
with open(json_file, 'r') as f:
    tickers = json.load(f)

# 为每个股票代码合并数据
for ticker in tickers:
    all_data = []  # 存储单个股票的所有数据
    # 遍历每个年份的文件夹
    for year in os.listdir(data_dir):
        year_dir = os.path.join(data_dir, year)
        if os.path.isdir(year_dir):  # 确保是目录
            # 假设后缀可能是.SZ或.SH，尝试两种可能性
            for suffix in ['.SZ', '.SH']:
                file_path = os.path.join(year_dir, f"{ticker}{suffix}.csv")
                if os.path.isfile(file_path):  # 确保文件存在
                    # 读取CSV文件的指定列
                    data = pd.read_csv(file_path)
                    all_data.append(data)
                    break  # 如果找到文件，则不需要尝试另一个后缀
    
    if all_data:
        # 合并数据
        merged_data = pd.concat(all_data)
        # 按交易时间排序
        merged_data.sort_values(by='trade_time', inplace=True)
        # 保存到merged文件夹
        merged_data.to_csv(os.path.join(merged_dir, f"{ticker}.csv"), index=False)

Merging Data: 100%|██████████| 696/696 [2:45:47<00:00, 14.29s/it]  


### 合并所有股票的列数据

In [9]:
import pandas as pd
import os
from tqdm import tqdm

def merge_csv_files(folder_path, output_folder, column):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    merged_data = pd.DataFrame()
    files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    for file in tqdm(files, desc='Merging files with column ' + column):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, parse_dates=['trade_time'])
        file_name = os.path.splitext(file)[0]
        df.rename(columns={column: file_name}, inplace=True)
        df = df[['trade_time', file_name]]
        
        if merged_data.empty:
            merged_data = df
        else:
            merged_data = pd.merge_ordered(merged_data, df, on='trade_time', fill_method='ffill')

    
    output_file = os.path.join(output_folder, f'merged_{column}.csv')
    merged_data.to_csv(output_file, index=False)


columns = [
#    'open',
#    'high', 
#    'low', 
    'close', 
#    'vol', 
    'amount'
    ]
folder_path = 'data/by_stock_merged_top_caps/'  
output_folder = 'data/merged_top_caps/'  

for column in columns:
    merge_csv_files(folder_path, output_folder, column)
    print(f'Merged {column} successfully!')


Merging files: 100%|██████████| 692/692 [49:35<00:00,  4.30s/it]


### 数据检查

切片前五列数据用于快速发现问题

In [1]:
import pandas as pd
file_path = 'data/merged_top_caps/merged_close.csv'
df = pd.read_csv(file_path)

# extract the first 5 columns
df = df.iloc[:, :5]
df.to_csv('data/merged_top_caps/merged_test.csv', index=False)

检查数据文件总行数

In [2]:
import pandas as pd
import os

# folder_path = 'data/merged_top_caps/'

# for filename in os.listdir(folder_path):
#     if filename.endswith('.csv'):
#         file_path = os.path.join(folder_path, filename)
#         df = pd.read_csv(file_path)
#         print(filename, df.shape)
#         print('\n')

file_path = 'data/merged_top_caps/merged_test.csv'
df = pd.read_csv(file_path)
print(df.shape)

(1398000, 5)


检查是否每天有240个数据点

In [3]:
import pandas as pd

file_path = 'data/merged_top_caps/merged_test.csv'
df = pd.read_csv(file_path)

# 检查是否每天有240个数据点
df['trade_time'] = pd.to_datetime(df['trade_time'])
df['date'] = df['trade_time'].dt.date
grouped = df.groupby('date').size()
result = grouped == 240
print(result)

invalid_dates = grouped[grouped != 240].index.tolist()
for i in invalid_dates:
    print(i, grouped[i])

# # list all the data points in trade_time column in the invalid dates
# invalid_data = df[df['date'].isin(invalid_dates)]
# print(invalid_data)



date
2000-01-04    True
2000-01-05    True
2000-01-06    True
2000-01-07    True
2000-01-10    True
              ... 
2024-01-08    True
2024-01-09    True
2024-01-10    True
2024-01-11    True
2024-01-12    True
Length: 5825, dtype: bool


### 处理某些日期多一个数据点的问题

In [1]:
import os
import pandas as pd

def process_csv_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            temp_file_path = os.path.join(folder_path, f"temp_{filename}")

            try:
                # 确定文件的总行数
                total_rows = sum(1 for row in open(file_path, 'r', encoding='utf-8'))

                # 读取除了最后10万行的所有数据
                read_rows = total_rows - 100000  
                data = pd.read_csv(file_path, nrows=read_rows)
                data.to_csv(temp_file_path, index=False)

                # 处理最后10万行
                data_last_part = pd.read_csv(file_path, skiprows=read_rows + 1, header=None)
                data_last_part.columns = data.columns  # 确保列名一致
                data_last_part = data_last_part[data_last_part['trade_time'] != '2023-11-30 13:00:00']

                # 如果最后10万行的trade_time数据有重复，删除重复的trade_time数据所在行
                data_last_part.drop_duplicates(subset=['trade_time'], inplace=True)

                # 将处理后的最后一部分追加到临时文件
                data_last_part.to_csv(temp_file_path, mode='a', index=False, header=False)

                # 替换原始文件
                os.remove(file_path)
                os.rename(temp_file_path, file_path)

                print(f"{filename}: 已完成处理")

            except Exception as e:
                print(f"处理文件 {filename} 时出错: {e}")

process_csv_files('data/merged_top_caps/')


merged_close.csv: 已完成处理


### 将分钟收盘价处理为收益率并后复权

In [7]:
import pandas as pd
import datetime
import os
from tqdm import tqdm

def read_data(file_path):
    df = pd.read_csv(file_path, index_col='trade_time')
    df.index = pd.to_datetime(df.index)  # 确保trade_time是DatetimeIndex类型
    return df

def process_period_data(df, period, minutes):
    if period == '1d':
        df_period = df.groupby(df.index.date).last()
    else:
        minutes_since_930 = (df.index.hour * 60 + df.index.minute) - (9 * 60 + 30)
        is_not_break_time = ~((df.index.hour == 11) & (df.index.minute > 30)) & ~((df.index.hour == 13) & (df.index.minute < 1))
        df_period = df[(minutes_since_930 % minutes == 0) & is_not_break_time]
    
    return df_period

def process_hfq_data(df_period_rate, hfq_data_path, ticker, minutes):
    file_found = False
    for prefix in ['sh', 'sz']:
        filename = f'{prefix}{ticker}.csv'
        file_path = os.path.join(hfq_data_path, filename)
        if os.path.exists(file_path):
            hfq_data = pd.read_csv(file_path)
            hfq_data['date'] = pd.to_datetime(hfq_data['date'])
            file_found = True
            break

    if file_found:
        for index, row in hfq_data[hfq_data['date'] > '2000-01-01'].iterrows():
            additional_hours, additional_minutes = divmod(30 + minutes, 60)
            trade_time = row['date'] + datetime.timedelta(hours=9 + additional_hours, minutes=additional_minutes)
            if trade_time in df_period_rate.index:
                df_period_rate.at[trade_time, ticker] = (df_period_rate.at[trade_time, ticker] + 1) * row['hfq_one_point'] - 1
    else:
        print(f'No hfq_one_point data file found for ticker {ticker}. Skipping...')

    return df_period_rate

# 主逻辑
file_path = 'data/merged_top_caps/merged_test.csv'
hfq_data_path = 'data/backward_adjust_factor'
df = read_data(file_path)

data_period = {
    '1d': 0,  # 特殊处理，0表示日数据
    '1m': 1,
    '5m': 5, 
    '15m': 15, 
    '60m': 60,
}

for period, minutes in data_period.items():
    df_period = process_period_data(df, period, minutes)
    df_period_rate = df_period.pct_change()
    df_period_rate.fillna(0, inplace=True)

    for ticker in tqdm(df_period_rate.columns, desc="Processing tickers"):
        df_period_rate = process_hfq_data(df_period_rate, hfq_data_path, ticker, minutes)

    df_period_rate.to_csv(f'data/period_rate/{period}.csv', index=True)
    df_period_rate.to_pickle(f'data/period_rate/{period}.pkl')
    print(f'Processed period {period}.')


# file_path = 'data/merged_top_caps/merged_test.csv'
# df = pd.read_csv(file_path, index_col='trade_time')
# df.index = pd.to_datetime(df.index)  # 确保trade_time是DatetimeIndex类型
# hfq_data_path = 'data/backward_adjust_factor'

# data_period = {
#     '1m': 1,
#     # '2m': 2, 
#     # '3m': 3,
#     '5m': 5, 
#     '15m': 15, 
#     # '30m': 30, 
#     '60m': 60,
# }

# for period, minutes in data_period.items():
#     if minutes == 1:
#         df_period = df.copy()
#     else:
#         minutes_since_930 = (df.index.hour * 60 + df.index.minute) - (9 * 60 + 30)
#         # 排除午休时间（11:30 - 13:00）
#         is_not_break_time = ~((df.index.hour == 11) & (df.index.minute > 30)) & ~((df.index.hour == 13) & (df.index.minute < 1))
#         df_period = df[(minutes_since_930 % minutes == 0) & is_not_break_time]

#     df_period_rate = df_period.pct_change()
#     df_period_rate.fillna(0, inplace=True)

#     for ticker in tqdm(df_period_rate.columns, desc="Processing tickers"):
#         file_found = False
#         for prefix in ['sh', 'sz']:
#             filename = f'{prefix}{ticker}.csv'
#             file_path = os.path.join(hfq_data_path, filename)
#             if os.path.exists(file_path):
#                 hfq_data = pd.read_csv(file_path)
#                 hfq_data['date'] = pd.to_datetime(hfq_data['date'])
#                 file_found = True
#                 break

#         if file_found:
#             for index, row in hfq_data[hfq_data['date'] > '2000-01-01'].iterrows():
#                 additional_hours, additional_minutes = divmod(30 + minutes, 60)
#                 trade_time = row['date'] + datetime.timedelta(hours=9 + additional_hours, minutes=additional_minutes)
#                 if trade_time in df_period_rate.index:
#                     df_period_rate.at[trade_time, ticker] = (df_period_rate.at[trade_time, ticker] + 1) * row['hfq_one_point'] - 1

#         else:
#             print(f'No hfq_one_point data file found for ticker {ticker}. Skipping...')

#     df_period_rate.to_csv(f'data/period_rate/{period}.csv', index=True)
#     df_period_rate.to_pickle(f'data/period_rate/{period}.pkl')
#     print(f'Processed period {period}.')


Processing tickers: 100%|██████████| 4/4 [00:00<00:00, 104.86it/s]


Processed period 1m.


Processing tickers: 100%|██████████| 4/4 [00:00<00:00, 82.82it/s]


Processed period 5m.


Processing tickers: 100%|██████████| 4/4 [00:00<00:00, 110.71it/s]


Processed period 15m.


Processing tickers: 100%|██████████| 4/4 [00:00<00:00, 114.61it/s]


Processed period 60m.


### 收益率数据检查

In [8]:
import os
import pandas as pd

folder_path = 'data/period_rate/'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, file), index_col=0)
    df.index = pd.to_datetime(df.index).date
    
    counts = df.groupby(df.index).size()
    
    if len(set(counts)) != 1:
        print(f'文件 {file} 的每天数据点数不同，请检查以下日期的数据：')
        for date, count in counts[counts != counts.mode()[0]].items():
            print(f'日期：{date}, 数据点数：{count}')

文件 15m.csv 的每天数据点数不同，请检查以下日期的数据：
日期：2023-11-30, 数据点数：15
文件 60m.csv 的每天数据点数不同，请检查以下日期的数据：
日期：2023-11-30, 数据点数：3


文件 5m.csv 的每天数据点数不同，请检查以下日期的数据：
日期：2023-11-30, 数据点数：47
